In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Cancernotes.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'Cancernotes.pdf', 'page': 0}, page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/338685968\nCancer notes\nChapt er · Januar y 2020\nCITATION\n1READS\n199,466\n1 author:\nMeena K umari\nAmity Univ ersity\n8 PUBLICA TIONS \xa0\xa0\xa045 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Meena K umari  on 19 Januar y 2020.\nThe user has r equest ed enhanc ement of the do wnlo aded file.'),
 Document(metadata={'source': 'Cancernotes.pdf', 'page': 1}, page_content="UNIT 1. NUTRITION IN CANCER  \n1 \nDr. Meena Kumari_Lecture Notes_Therapeutic Nutrition  \nAmity Institute of Food Technology, AMITY University     Email Id: meena3020@gmail.com      Nutrition in cancer  \n• Pathogenesis of cancer  \n• Causes of cancer cell development  \n• Metabolic and nutritional alterations in malignancy  \n• Cancer therapy  \n• Nutrition & eating problems in 

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:20]

[Document(metadata={'source': 'Cancernotes.pdf', 'page': 0}, page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/338685968\nCancer notes\nChapt er · Januar y 2020\nCITATION\n1READS\n199,466\n1 author:\nMeena K umari\nAmity Univ ersity\n8 PUBLICA TIONS \xa0\xa0\xa045 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Meena K umari  on 19 Januar y 2020.\nThe user has r equest ed enhanc ement of the do wnlo aded file.'),
 Document(metadata={'source': 'Cancernotes.pdf', 'page': 1}, page_content='UNIT 1. NUTRITION IN CANCER  \n1 \nDr. Meena Kumari_Lecture Notes_Therapeutic Nutrition  \nAmity Institute of Food Technology, AMITY University     Email Id: meena3020@gmail.com      Nutrition in cancer  \n• Pathogenesis of cancer  \n• Causes of cancer cell development  \n• Metabolic and nutritional alterations in malignancy  \n• Cancer therapy  \n• Nutrition & eating problems in 

In [6]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'Cancernotes.pdf', 'page': 0}, page_content='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/338685968\nCancer notes\nChapt er · Januar y 2020\nCITATION\n1READS\n199,466\n1 author:\nMeena K umari\nAmity Univ ersity\n8 PUBLICA TIONS \xa0\xa0\xa045 CITATIONS \xa0\xa0\xa0\nSEE PROFILE\nAll c ontent f ollo wing this p age was uplo aded b y Meena K umari  on 19 Januar y 2020.\nThe user has r equest ed enhanc ement of the do wnlo aded file.'),
 Document(metadata={'source': 'Cancernotes.pdf', 'page': 1}, page_content='UNIT 1. NUTRITION IN CANCER  \n1 \nDr. Meena Kumari_Lecture Notes_Therapeutic Nutrition  \nAmity Institute of Food Technology, AMITY University     Email Id: meena3020@gmail.com      Nutrition in cancer  \n• Pathogenesis of cancer  \n• Causes of cancer cell development  \n• Metabolic and nutritional alterations in malignancy  \n• Cancer therapy  \n• Nutrition & eating problems in 

In [14]:

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30],OllamaEmbeddings())

In [17]:
query="cancer is based around tumours? "
result=db.similarity_search(query)
result[0].page_content

'UNIT 1. NUTRITION IN CANCER  \n1 \nDr. Meena Kumari_Lecture Notes_Therapeutic Nutrition  \nAmity Institute of Food Technology, AMITY University     Email Id: meena3020@gmail.com      Nutrition in cancer  \n• Pathogenesis of cancer  \n• Causes of cancer cell development  \n• Metabolic and nutritional alterations in malignancy  \n• Cancer therapy  \n• Nutrition & eating problems in cancer.  \n \nWhat is cancer?  \nCancer is the uncontrolled growth of abnormal cells anywhere in the body. These abnormal cells \nare termed cancer cells,  malignant  cells, or  tumor  cells. These cells can infiltrate normal body \ntissues. Many cancers and the abnormal cells that compose the ca ncer tissue are further identified \nby the name of the tissue that the abnormal cells originated from (for example,  breast \ncancer,  lung cancer,  and colorectal  cancer ). When damaged or unrepaired cells do not die and \nbecome cancer cells and show uncontrolled  division and growth - a mass of cancer cells deve

In [15]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [18]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [19]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [20]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000012727246B50>)

In [21]:

"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [22]:

response=retrieval_chain.invoke({"input":"Objectives of nutritional therapy"})

In [23]:
response['answer']

"Based on the provided context, the objectives of nutritional therapy for cancer patients are to:\n\n1. Address nutritional problems related to surgical treatment, such as poor wound healing and reduced immune function.\n2. Help manage side effects of radiotherapy, including nausea, vomiting, and diarrhea.\n3. Minimize the risk of developing nutrient deficiencies during chemotherapy, which can lead to fatigue, infection, and other complications.\n4. Promote a healthy diet and lifestyle to support the body's immune system and overall health during cancer treatment.\n5. Address any specific nutritional needs or requirements of the patient, such as managing blood sugar levels or ensuring adequate protein intake.\n6. Help patients cope with the emotional and psychological aspects of cancer treatment through nutrition counseling and support.\n7. Monitor and manage weight changes during cancer treatment to prevent undesirable side effects, such as fatigue and weakness.\n8. Address any malnut